In [2]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import requests
import time

import pandas as pd
import datetime

In [2]:
from selenium.webdriver.edge.options import Options
edge_options = Options()
edge_options.add_experimental_option("detach", True)
edge_options.add_argument("--remote-debugging-port=9222") 
edge_options.add_experimental_option('useAutomationExtension', False) 
edge_options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [3]:
driver = webdriver.Edge(options=edge_options)
driver.maximize_window()

In [4]:
url='https://twitter.com/home'
driver.get(url)
time.sleep(2)

In [5]:
driver.find_element(By.CSS_SELECTOR,'div[class="css-175oi2r r-16y2uox r-1wbh5a2 r-1jgb5lz r-13qz1uu r-1ye8kvj"] div[class="css-175oi2r r-ywje51 r-usiww2 r-eu3ka r-1ipicw7"]').click()

In [6]:
window_handles = driver.window_handles
if len(window_handles) > 1:
    driver.switch_to.window(window_handles[1])
time.sleep(2)

In [7]:
sign_in_box=driver.find_element(By.CSS_SELECTOR,'div[class="Xb9hP"] input[class="whsOnd zHQkBf"]')
sign_in_box.clear()
time.sleep(2)
sign_in_box.click()

text_to_paste='myemail.gmail.com'
sign_in_box.send_keys(text_to_paste)
sign_in_box.send_keys(Keys.RETURN)
time.sleep(2)

In [8]:
sign_in_box=driver.find_element(By.CSS_SELECTOR,'div[class="Xb9hP"] input[class="whsOnd zHQkBf"]')
sign_in_box.clear()
time.sleep(2)
sign_in_box.click()

text_to_paste='mypassword'
sign_in_box.send_keys(text_to_paste)
sign_in_box.send_keys(Keys.RETURN)
time.sleep(2)

In [9]:
driver.switch_to.window(window_handles[0])
time.sleep(2)

In [10]:
try:
    driver.find_element(By.CSS_SELECTOR,'div[class="css-175oi2r r-13qz1uu"] div[role="button"]').click()
except:
    None

In [11]:
"""search_bar=WebDriverWait(driver,10).until(expected_conditions.visibility_of_element_located((By.CSS_SELECTOR,'div[class="css-175oi2r r-aqfbo4 r-gtdqiz r-1gn8etr r-1g40b8q"] input')))
search_bar.clear()
time.sleep(2)
search_bar.click()

text_to_paste='#trump'
search_bar.send_keys(text_to_paste)
search_bar.send_keys(Keys.RETURN)
time.sleep(2)"""

'search_bar=WebDriverWait(driver,10).until(expected_conditions.visibility_of_element_located((By.CSS_SELECTOR,\'div[class="css-175oi2r r-aqfbo4 r-gtdqiz r-1gn8etr r-1g40b8q"] input\')))\nsearch_bar.clear()\ntime.sleep(2)\nsearch_bar.click()\n\ntext_to_paste=\'#trump\'\nsearch_bar.send_keys(text_to_paste)\nsearch_bar.send_keys(Keys.RETURN)\ntime.sleep(2)'

In [12]:
driver.get('https://twitter.com/search?q=%23trump&src=typed_query')

In [13]:
actions=ActionChains(driver)
date_scraped=pd.to_datetime(datetime.date.today()).strftime('%Y-%m-%d')

data=[]
for _ in range(31):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    elements=driver.find_elements(By.CSS_SELECTOR,'div[data-testid="cellInnerDiv"]')

    for element in elements:
        try:
            author_and_date=element.find_element(By.CSS_SELECTOR,'div[class="css-175oi2r r-zl2h9q"]').get_attribute('textContent').replace('\n','')
        except:
            author_and_date=None

        try:
            text=element.find_element(By.CSS_SELECTOR,'div[data-testid="tweetText"]').get_attribute('textContent').replace('\n','')
        except:
            text=None

        try:
            engagement=[i.text if i else None for i in element.find_elements(By.CSS_SELECTOR,'div[class="css-175oi2r r-1kbdv8c r-18u37iz r-1wtj0ep r-1ye8kvj r-1s2bzr4"] div[class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"]')]
        except:
            engagement=None

        data.append({'Author And Date':author_and_date,'Text':text,'Engagement':engagement,'Date Scraped':date_scraped})
        print(f'Tweets: {len(data)}, Arrows Down: {_}',end='\r')

In [14]:
df=pd.DataFrame(data).drop_duplicates(subset=['Author And Date','Text']).dropna(how='all')
df

,Author And Date,Text,Engagement,Date Scraped
0,OverVechter@112_overvecht·3h,Democrats finally become nazis!#Biden #trump #...,"[2, 6, 7, 750]",2023-12-30
1,DARKSEID_BC_@BcDarkseid·Dec 23,Witnesses to #Trump crapping his pants in fron...,"[103, 1K, 1.8K, 140K]",2023-12-30
2,No Lies Pay it Forward@nolietees·Dec 28,#TRUMP2024ToSaveAmericaMore like #Trump2024ToF...,"[20, 57, 114, 1.6K]",2023-12-30
3,GIFT OF THE GAB@giftgab538·Dec 23,#Elections2024 #Democracy #Elon #Trump #Ronn...,"[80, 983, 1.2K, 23K]",2023-12-30
4,Brandon Taylor Moore@LetsGoBrando45·Dec 24·,You’re ignorant to the facts if you believe #T...,"[12, 41, 65, 3K]",2023-12-30
...,...,...,...,...
195,True Blue @SPBMcKenna·Dec 26,"#Trump not only suffers from INCONTINENCE, he...","[20, 48, 88, 2K]",2023-12-30
200,Politics@Politics15419·Dec 28,"Win with DeSantis, lose with Trump.#Trump is f...","[1, , 1, 30]",2023-12-30
201,No Lies Pay it Forward@nolietees·Dec 26,"Yes, we remember the ""Trump Healthcare"" book h...","[1, 7, 14, 184]",2023-12-30
202,Scamnomics@scamnomics9·Dec 23,THE PLUNGE PROTECTION FUND ARE TRYING TO SAVE ...,"[2, 2, 1, 3.1K]",2023-12-30


In [15]:
#df.to_csv('csv_files/X.csv',index=False)

In [16]:
#df=pd.read_csv('csv_files/X.csv')

### Cleaning Dataset

In [ ]:
df.insert(0,'User Name',df['Author And Date'].apply(lambda x:'@'+x.split('·')[0].split('@')[-1] if isinstance(x,str) else None))

In [ ]:
df=df['Author And Date'].apply(lambda x:x.split('·')[-1] if isinstance(x,str) else None)

today=pd.to_datetime(datetime.date.today()).strftime("%b %d, %Y")
df['Post Date']=df['Post Date'].apply(lambda x:x+', 2023' if ' ' in str(x) else today)

In [24]:
df=df.drop(columns='Author And Date')

In [28]:
import ast
def ast_(str):
    try:
        return ast.literal_eval(str)
    except:
        return None

df['Engagement']=df['Engagement'].apply(ast_)

In [34]:
engagement_dict_list=[]
for engagement in df['Engagement']:
    if isinstance(engagement,list):
        engagement_dict={'Comments':engagement[0],'Reposts':engagement[1],'Likes':engagement[2],'Views':engagement[3]}
        engagement_dict_list.append(engagement_dict)
    else:
        engagement_dict_list.append(None)

In [36]:
df['Engagement']=engagement_dict_list

In [26]:
df.to_csv('csv_files/X.csv',index=False)

In [27]:
df=pd.read_csv('csv_files/X.csv')

In [28]:
df

,User Name,Text,Engagement,Date Scraped,Post Date
0,@112_overvecht,Democrats finally become nazis!#Biden #trump #...,"{'Comments': '2', 'Reposts': '6', 'Likes': '7'...",2023-12-30,"Dec 30, 2023"
1,@BcDarkseid,Witnesses to #Trump crapping his pants in fron...,"{'Comments': '103', 'Reposts': '1K', 'Likes': ...",2023-12-30,"Dec 23, 2023"
2,@nolietees,#TRUMP2024ToSaveAmericaMore like #Trump2024ToF...,"{'Comments': '20', 'Reposts': '57', 'Likes': '...",2023-12-30,"Dec 28, 2023"
3,@giftgab538,#Elections2024 #Democracy #Elon #Trump #Ronn...,"{'Comments': '80', 'Reposts': '983', 'Likes': ...",2023-12-30,"Dec 23, 2023"
4,@LetsGoBrando45,You’re ignorant to the facts if you believe #T...,"{'Comments': '12', 'Reposts': '41', 'Likes': '...",2023-12-30,"Dec 30, 2023"
...,...,...,...,...,...
174,@SPBMcKenna,"#Trump not only suffers from INCONTINENCE, he...","{'Comments': '20', 'Reposts': '48', 'Likes': '...",2023-12-30,"Dec 26, 2023"
175,@Politics15419,"Win with DeSantis, lose with Trump.#Trump is f...","{'Comments': '1', 'Reposts': '', 'Likes': '1',...",2023-12-30,"Dec 28, 2023"
176,@nolietees,"Yes, we remember the ""Trump Healthcare"" book h...","{'Comments': '1', 'Reposts': '7', 'Likes': '14...",2023-12-30,"Dec 26, 2023"
177,@scamnomics9,THE PLUNGE PROTECTION FUND ARE TRYING TO SAVE ...,"{'Comments': '2', 'Reposts': '2', 'Likes': '1'...",2023-12-30,"Dec 23, 2023"
